In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [4]:
#in real world, this would be an API call to a stock price service
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the symbol
    :param symbol: stock symbol
    :return: current price of the stock
    '''
    return {
        "MSFT": 300.0,
        "AAPL": 150.0,
        "AMZN": 3500.0,
        "RIL": 2000.0,
    }.get(symbol, 0.0)

tools = [get_stock_price]

llm = init_chat_model("llama-3.3-70b-versatile",model_provider="groq")
llm_with_tools = llm.bind_tools(tools)

In [5]:
def chatbot(state: State) -> State:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot_node")
builder.add_conditional_edges("chatbot_node", tools_condition)
builder.add_edge("tools", "chatbot_node")
# builder.add_edge(tools_condition, "tools")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [6]:
from IPython.display import Markdown
Markdown(f"```mermaid\n{graph.get_graph().draw_mermaid()}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot_node(chatbot_node)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot_node;
	chatbot_node -.-> __end__;
	chatbot_node -.-> tools;
	tools --> chatbot_node;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [7]:
from langsmith import traceable

@traceable
def call_graph(query: str):
    state = graph.invoke({"messages": [{"role": "user", "content": query}]})
    return state["messages"][-1].content

call_graph("I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be total cost?")

'The total cost of buying 20 AMZN stocks and 15 MSFT stocks can be calculated by multiplying the number of stocks by the current price of each stock.\n\nFor AMZN: 20 * $3500.0 = $70,000.0\nFor MSFT: 15 * $300.0 = $4,500.0\n\nThe total cost is: $70,000.0 + $4,500.0 = $74,500.0'

In [8]:
state

NameError: name 'state' is not defined